In [40]:
import warnings
warnings.filterwarnings('ignore')


In [41]:

from nltk.corpus import stopwords
import nltk
from nltk.stem.porter import PorterStemmer

import pandas as pd
%matplotlib inline

from collections import OrderedDict
from pathlib import Path

import numpy as np
import pandas as pd

# Visualization
import matplotlib.pyplot as plt
from matplotlib.ticker import FuncFormatter
import seaborn as sns

import pyLDAvis
from pyLDAvis.sklearn import prepare

from wordcloud import WordCloud
from termcolor import colored

# sklearn for feature extraction & modeling
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.model_selection import train_test_split
import joblib

import re

%matplotlib inline
import sys
from pathlib import Path

import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import GridSearchCV
import pandas
from sklearn import model_selection
from sklearn.linear_model import LogisticRegression
import pickle
import spacy
from spacy import displacy
from textacy.extract import ngrams, entities
nlp = spacy.load('en_core_web_lg') 
import os

In [42]:
# change to your data path if necessary
DATA_DIR = Path('./data/')
data_path = DATA_DIR / 'earnings_calls'
results_path = Path('results')
model_path = Path('results', 'earnings_calls')
if not model_path.exists():
    model_path.mkdir(exist_ok=True, parents=True)
    
    
files = sorted(list(data_path.glob('**/*.csv')))
doc_list = []
for i  in (files):
    with open(str(i), encoding='latin1') as f:
        topic = i.parts[-2]
        lines = f.readlines()
        heading = lines[0].strip()
        body = ' '.join([l.strip() for l in lines[1:]])
        doc_list.append([topic.capitalize(), heading, body])
        
Dfdocs = pd.DataFrame(doc_list, columns=['Ticker', 'heading', 'article'])




In [43]:

datadf = pd.DataFrame(pd.read_csv('movie_data.csv'),
                      columns=['review', 'sentiment'])


In [44]:
import numpy as np
import re
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer

stop = stopwords.words('english')
porter = PorterStemmer()

def tokenizer(text):
    text = re.sub('<[^>]*>', '', text)
    emoticons = re.findall('(?::|;|=)(?:-)?(?:\)|\(|D|P)', text.lower())
    text = re.sub('[\W]+', ' ', text.lower()) + ' '.join(emoticons).replace('-', '')
    tokenized = [w for w in text.split() if w not in stop]
    return tokenized

def stream_docs(path):
    with open(path, 'r', encoding='utf-8') as csv:
        next(csv) # skip header
        for line in csv:
            text, label = line[:-3], int(line[-2])
            yield text, label
            

def get_minibatch(doc_stream, size):
    docs, y = [], []
    try:
        for _ in range(size):
            text, label = next(doc_stream)
            docs.append(text)
            y.append(label)
    except StopIteration:
        return None, None
    return docs, y            




import re
def preprocessor(text):
    text = re.sub('<[^>]*>', '', text)
    emoticons = re.findall('(?::|;|=)(?:-)?(?:\)|\(|D|P)',
                           text)
    text = (re.sub('[\W]+', ' ', text.lower()) +
            ' '.join(emoticons).replace('-', ''))
    return text

procdoc_list = preprocessor(str(Dfdocs))
                            
display(procdoc_list , next(stream_docs(path='movie_data.csv'))  )

' ticker heading 0 a speaker q a content 1 a company agilent technologies inc s 2 a type name 3 abb speaker q a content 4 abb company abb s 2125 zuo company zuora inc 2126 zuo type name 2127 zx speaker q a content 2128 zx company china zenix s 2129 zx type name article 0 operator 0 good day ladies and gentlemen an 1 day 25 link a class dashboard article link 2 executives alicia rodriguez vice president o 3 company participants 0 jessica mitchell â h 4 symbol abb month 4 day 17 year 19 quarter q1 l 2125 symbol zuo month 5 day 30 year 19 link a cla 2126 2127 operator 0 greetings and welcome to the chin 2128 day 8 link a class dashboard article link 2129 executives kevin theiss ir executives martin 2130 rows x 3 columns '

('"In 1974, the teenager Martha Moxley (Maggie Grace) moves to the high-class area of Belle Haven, Greenwich, Connecticut. On the Mischief Night, eve of Halloween, she was murdered in the backyard of her house and her murder remained unsolved. Twenty-two years later, the writer Mark Fuhrman (Christopher Meloni), who is a former LA detective that has fallen in disgrace for perjury in O.J. Simpson trial and moved to Idaho, decides to investigate the case with his partner Stephen Weeks (Andrew Mitchell) with the purpose of writing a book. The locals squirm and do not welcome them, but with the support of the retired detective Steve Carroll (Robert Forster) that was in charge of the investigation in the 70\'s, they discover the criminal and a net of power and money to cover the murder.<br /><br />""Murder in Greenwich"" is a good TV movie, with the true story of a murder of a fifteen years old girl that was committed by a wealthy teenager whose mother was a Kennedy. The powerful and rich f

In [45]:


Dfdocs[['Ticker', 'article']]

inline = Dfdocs.set_index('Ticker').drop(columns='heading')

In [46]:
inline['article'] = inline['article'].apply(preprocessor)


In [47]:
from sklearn.feature_extraction.text import HashingVectorizer
from sklearn.linear_model import SGDClassifier

vect = HashingVectorizer(decode_error='ignore', 
                         n_features=2**21,
                         preprocessor=None, 
                         tokenizer=tokenizer)

clf = SGDClassifier(loss='log', random_state=1, max_iter=1)
doc_stream = stream_docs(path='movie_data.csv')

In [48]:
import pyprind
pbar = pyprind.ProgBar(45)

classes = np.array([0, 1])
for _ in range(45):
    X_train, y_train = get_minibatch(doc_stream, size=1000)
    if not X_train:
        break
    X_train = vect.transform(X_train)
    clf.partial_fit(X_train, y_train, classes=classes)
    pbar.update()

0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:00:33


In [49]:
X_test, y_test = get_minibatch(doc_stream, size=5000)
X_test = vect.transform(X_test)
print('Accuracy: %.3f' % clf.score(X_test, y_test))

Accuracy: 0.868


In [50]:
clf = clf.partial_fit(X_test, y_test)

In [51]:
import os 

dest = os.path.join('movieclassifier', 'pkl_objects')
if not os.path.exists(dest):
    os.makedirs(dest)

pickle.dump(stop, open(os.path.join(dest, 'stopwords.pkl'), 'wb'), protocol=4)   
pickle.dump(clf, open(os.path.join(dest, 'classifier.pkl'), 'wb'), protocol=4)

In [52]:
%%writefile movieclassifier/vectorizer.py
from sklearn.feature_extraction.text import HashingVectorizer
import re
import os
import pickle

cur_dir = os.path.dirname(__file__)
stop = pickle.load(open(
                os.path.join(cur_dir, 
                'pkl_objects', 
                'stopwords.pkl'), 'rb'))

def tokenizer(text):
    text = re.sub('<[^>]*>', '', text)
    emoticons = re.findall('(?::|;|=)(?:-)?(?:\)|\(|D|P)',
                           text.lower())
    text = re.sub('[\W]+', ' ', text.lower()) \
                   + ' '.join(emoticons).replace('-', '')
    tokenized = [w for w in text.split() if w not in stop]
    return tokenized

vect = HashingVectorizer(decode_error='ignore',
                         n_features=2**21,
                         preprocessor=None,
                         tokenizer=tokenizer)

Overwriting movieclassifier/vectorizer.py


In [54]:
os.chdir('movieclassifier')

In [55]:
import pickle
import re
import os
from vectorizer import vect

clf = pickle.load(open(os.path.join('pkl_objects', 'classifier.pkl'), 'rb'))


In [57]:
import numpy as np
label = {0:'negative', 1:'positive'}

X = vect.transform(str(inline.loc["A"].values))
print('Prediction: %s\nProbability: %.2f%%' %\
      (label[clf.predict(X)[0]], 
       np.max(clf.predict_proba(X))*100))

ValueError: Iterable over raw text documents expected, string object received.

In [ ]:
datadf['review'] = datadf['review'].apply(preprocessor)

X_train = datadf.loc[:25000, 'review'].values
y_train = datadf.loc[:25000, 'sentiment'].values
X_test = datadf.loc[25000:, 'review'].values
y_test = datadf.loc[25000:, 'sentiment'].values

In [ ]:

tfidf = TfidfVectorizer(strip_accents=None,
                        lowercase=False,
                        preprocessor=None)

param_grid = [{'vect__ngram_range': [(1, 1)],
               'vect__stop_words': [stop, None],
               'vect__tokenizer': [tokenizer, tokenizer_porter],
               'clf__penalty': ['l1', 'l2'],
               'clf__C': [1.0, 10.0, 100.0]},
              {'vect__ngram_range': [(1, 1)],
               'vect__stop_words': [stop, None],
               'vect__tokenizer': [tokenizer, tokenizer_porter],
               'vect__use_idf':[False],
               'vect__norm':[None],
               'clf__penalty': ['l1', 'l2'],
               'clf__C': [1.0, 10.0, 100.0]},
              ]

lr_tfidf = Pipeline([('vect', tfidf),
                     ('clf', LogisticRegression(random_state=0, solver='liblinear'))])

gs_lr_tfidf = GridSearchCV(lr_tfidf, param_grid,
                           scoring='accuracy',
                           cv=5,
                           verbose=2,
                           n_jobs=-1)

In [ ]:
#gs_lr_tfidf.fit(X_train, y_train)


In [ ]:

print('Best parameter set: %s ' % gs_lr_tfidf.best_params_)
print('CV Accuracy: %.3f' % gs_lr_tfidf.best_score_)

In [ ]:
clf = gs_lr_tfidf.best_estimator_
print('Test Accuracy: %.3f' % clf.score(X_test, y_test))


In [ ]:
import pickle
import os

dest = os.path.join('movieclassifier', 'pkl_objects')
if not os.path.exists(dest):
    os.makedirs(dest)


In [ ]:

pickle.dump(stop, open(os.path.join(dest, 'stopwords.pkl'), 'wb'), protocol=4)   
pickle.dump(clf, open(os.path.join(dest, 'classifier.pkl'), 'wb'), protocol=4)

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer


In [ ]:

datadf = pd.DataFrame(pd.read_csv('movie_data.csv'),
                      columns=['review', 'sentiment'])


In [ ]:
loaded_model = pickle.load(open('movieclassifier/pkl_objects/classifier.pkl'))
result = loaded_model.score(X_test, y_test)
print(result)

In [ ]:

count = CountVectorizer(stop_words='english',
                        max_features=500)
X = count.fit_transform(datadf['review'].values)


In [ ]:
#%%writefile movieclassifier/vectorizer.py
from sklearn.feature_extraction.text import HashingVectorizer
import re
import os
import pickle

cur_dir = os.path.dirname(__file__)
stop = pickle.load(open(
                os.path.join(cur_dir, 
                'pkl_objects', 
                'stopwords.pkl'), 'rb'))


In [ ]:
import os


vect = HashingVectorizer(decode_error='ignore',
                         n_features=2**21,
                         preprocessor=None,
                         tokenizer=tokenizer)

os.chdir('movieclassifier')

In [ ]:
# change to your data path if necessary
DATA_DIR = Path('./data/')
data_path = DATA_DIR / 'earnings_calls'
results_path = Path('results')
model_path = Path('results', 'earnings_calls')
if not model_path.exists():
    model_path.mkdir(exist_ok=True, parents=True)
    

In [ ]:
files = pd.read_csv(list(data_path.glob('**/*.csv')))
doc_list = []
for i  in (files):
    with open(str(i), encoding='latin1') as f:
        topic = i.parts[-2]
        lines = f.readlines()
        heading = lines[0].strip()
        body = ' '.join([l.strip() for l in lines[1:]])
        doc_list.append([topic.capitalize(), heading, body])

In [ ]:
Dfdocs = pd.DataFrame(doc_list, columns=['Ticker', 'heading', 'article'])


In [ ]:
Dfdocs

In [ ]:
import numpy as np
label = {0:'negative', 1:'positive'}

example = ("I love this movie. It's amazing.",
           'Happy life')
X = vect.transform(example)
print('Prediction: %s\nProbability: %.2f%%' %\
      (label[clf.predict(X)[0]], 
       np.max(clf.predict_proba(X))*100))